<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#DATA-PREPROCESSING" data-toc-modified-id="DATA-PREPROCESSING-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>DATA PREPROCESSING</a></span><ul class="toc-item"><li><span><a href="#Import-relevant-libraries:" data-toc-modified-id="Import-relevant-libraries:-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Import relevant libraries:</a></span></li><li><span><a href="#Importing-Dataset-and-Evaluating-the-Datastructure" data-toc-modified-id="Importing-Dataset-and-Evaluating-the-Datastructure-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Importing Dataset and Evaluating the Datastructure</a></span></li></ul></li><li><span><a href="#Creating-a-Training-and-Validation-Set" data-toc-modified-id="Creating-a-Training-and-Validation-Set-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Creating a Training and Validation Set</a></span></li><li><span><a href="#Implementing-Alternating-Least-Squares(ALS)-for-Implicit-Feedback" data-toc-modified-id="Implementing-Alternating-Least-Squares(ALS)-for-Implicit-Feedback-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Implementing Alternating Least Squares(ALS) for Implicit Feedback</a></span><ul class="toc-item"><li><span><a href="#Speeding-Up-ALS" data-toc-modified-id="Speeding-Up-ALS-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Speeding Up ALS</a></span></li></ul></li><li><span><a href="#Evaluating-the-Recommender-System" data-toc-modified-id="Evaluating-the-Recommender-System-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Evaluating the Recommender System</a></span><ul class="toc-item"><li><span><a href="#An-Example-using-our-recommendation-engine" data-toc-modified-id="An-Example-using-our-recommendation-engine-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>An Example using our recommendation engine</a></span></li><li><span><a href="#CHOOSING-A-RANDOM-LIST-OF-10-CUSTOMERS-TO-RECOMMEND-PRODUCTS-FOR-THEM-USING-OUR-MODEL" data-toc-modified-id="CHOOSING-A-RANDOM-LIST-OF-10-CUSTOMERS-TO-RECOMMEND-PRODUCTS-FOR-THEM-USING-OUR-MODEL-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>CHOOSING A RANDOM LIST OF 10 CUSTOMERS TO RECOMMEND PRODUCTS FOR THEM USING OUR MODEL</a></span></li></ul></li><li><span><a href="#Importing-Price-and-Cost-data-for-products" data-toc-modified-id="Importing-Price-and-Cost-data-for-products-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Importing Price and Cost data for products</a></span><ul class="toc-item"><li><span><a href="#Merging-the-price-and-cost-to-our-Recommendations" data-toc-modified-id="Merging-the-price-and-cost-to-our-Recommendations-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Merging the price and cost to our Recommendations</a></span></li><li><span><a href="#Profit-on-the-Recommended-Products-of-10-Random-customers" data-toc-modified-id="Profit-on-the-Recommended-Products-of-10-Random-customers-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>Profit on the Recommended Products of 10 Random customers</a></span></li></ul></li><li><span><a href="#Estimated-Total-Revenue/Profit-for-recommended-products-to-all-customers-if-purchased" data-toc-modified-id="Estimated-Total-Revenue/Profit-for-recommended-products-to-all-customers-if-purchased-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Estimated Total Revenue/Profit for recommended products to all customers if purchased</a></span></li></ul></div>

# DATA PREPROCESSING

### Import relevant libraries:
**Import all the relevant python libraries for building supervised machine learning algorithms.**

In [1]:
import pandas as pd
import scipy.sparse as sparse
import numpy as np
import statistics
from scipy.sparse.linalg import spsolve
from pandas.api.types import CategoricalDtype
import pyodbc

### Importing Dataset and Evaluating the Datastructure

In [2]:
# connect to SQL (1st step)

# establish an open connection to SQL
conn = pyodbc.connect('Driver={SQL Server};'
'Server=DESKTOP-90M94KF;'
'Database=AdventureWorksDW2019;'
'Trusted_Connection=yes;')

In [3]:
my_data=pd.read_sql_query("""
SELECT 
[SalesOrderNumber] AS Invoice_NO
,[SalesOrderLineNumber] AS Invoice_Line_NO
,[OrderDate] AS Invoice_Date
,CST.CustomerKey AS Customer_ID
,CONCAT(CST.FirstName, ' ', CST.LastName) AS Customer_Name
,[ModelName] AS Product
,PR.[EnglishProductName] AS Product_Name
,PR.[ProductKey] AS Product_ID
,[OrderQuantity] AS Quantity
,PRS.EnglishProductSubcategoryName AS Sub_Category

FROM [dbo].[FactInternetSales] AS FS

JOIN [dbo].[DimCustomer] CST

ON FS.CustomerKey = CST.CustomerKey

JOIN [dbo].[DimProduct] AS PR

ON FS.ProductKey = PR.ProductKey

JOIN [dbo].[DimProductSubcategory] AS PRS

ON PR.ProductSubcategoryKey = PRS.ProductSubcategoryKey
""", conn)

conn.close()

In [4]:
my_data.head()

,Invoice_NO,Invoice_Line_NO,Invoice_Date,Customer_ID,Customer_Name,Product,Product_Name,Product_ID,Quantity,Sub_Category
0,SO43697,1,2010-12-29,21768,Cole Watson,Road-150,"Road-150 Red, 62",310,1,Road Bikes
1,SO43698,1,2010-12-29,28389,Rachael Martinez,Mountain-100,"Mountain-100 Silver, 44",346,1,Mountain Bikes
2,SO43699,1,2010-12-29,25863,Sydney Wright,Mountain-100,"Mountain-100 Silver, 44",346,1,Mountain Bikes
3,SO43700,1,2010-12-29,14501,Ruben Prasad,Road-650,"Road-650 Black, 62",336,1,Road Bikes
4,SO43701,1,2010-12-29,11003,Christy Zhu,Mountain-100,"Mountain-100 Silver, 44",346,1,Mountain Bikes


In [5]:
my_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60398 entries, 0 to 60397
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Invoice_NO       60398 non-null  object        
 1   Invoice_Line_NO  60398 non-null  int64         
 2   Invoice_Date     60398 non-null  datetime64[ns]
 3   Customer_ID      60398 non-null  int64         
 4   Customer_Name    60398 non-null  object        
 5   Product          60398 non-null  object        
 6   Product_Name     60398 non-null  object        
 7   Product_ID       60398 non-null  int64         
 8   Quantity         60398 non-null  int64         
 9   Sub_Category     60398 non-null  object        
dtypes: datetime64[ns](1), int64(4), object(5)
memory usage: 4.6+ MB


The Dataset includes the invoice number for different purchases, along with the Product ID, the Sub-Category of Products, Product model and the Product Name, the quantity purchased, the Invoice Date, a customer ID and Name of each customer.

Let’s check to see if there are any missing values in the data.

In [6]:
my_data.nunique()

Invoice_NO         27659
Invoice_Line_NO        8
Invoice_Date        1124
Customer_ID        18484
Customer_Name      18400
Product               40
Product_Name         130
Product_ID           158
Quantity               1
Sub_Category          17
dtype: int64

In [7]:
my_data.isnull().sum()

Invoice_NO         0
Invoice_Line_NO    0
Invoice_Date       0
Customer_ID        0
Customer_Name      0
Product            0
Product_Name       0
Product_ID         0
Quantity           0
Sub_Category       0
dtype: int64

No missing values in our Dataset

In [8]:
# Lets have a lookup table that keeps track of each Product ID along with a description of that product.

item_lookup = my_data[['Product_ID', 'Product', 'Product_Name']].drop_duplicates() # Only get unique Product/Product Name pairs
item_lookup['Product_ID'] = item_lookup.Product_ID.astype(str) # Encode as strings for future lookup ease

In [9]:
item_lookup.head()

,Product_ID,Product,Product_Name
0,310,Road-150,"Road-150 Red, 62"
1,346,Mountain-100,"Mountain-100 Silver, 44"
3,336,Road-650,"Road-650 Black, 62"
5,311,Road-150,"Road-150 Red, 44"
7,351,Mountain-100,"Mountain-100 Black, 48"


In [10]:
# Group purchase quantities together by Customer ID and Product ID

my_data['Customer_ID'] = my_data.Customer_ID.astype(int) # Convert to int for customer_ID
my_data = my_data[['Product_ID', 'Quantity', 'Customer_ID']] # Get rid of unnecessary info
grouped_retail = my_data.groupby(['Customer_ID', 'Product_ID']).sum().reset_index() # Group together
grouped_retail.Quantity.loc[grouped_retail.Quantity == 0] = 1 # Replace a sum of zero purchases with a one to

# indicate purchased
grouped_purchased = grouped_retail.query('Quantity > 0') # Only get customers where purchase totals were positive

In [11]:
grouped_purchased.head()

,Customer_ID,Product_ID,Quantity
0,11000,214,1
1,11000,344,1
2,11000,353,1
3,11000,485,1
4,11000,488,1


In [12]:
grouped_purchased['Product_ID'] = grouped_purchased['Product_ID'].astype(str) #convert ProductID to string

In [13]:
grouped_purchased.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59051 entries, 0 to 59050
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Customer_ID  59051 non-null  int64 
 1   Product_ID   59051 non-null  object
 2   Quantity     59051 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 1.8+ MB


In [14]:
customers = list(np.sort(grouped_purchased.Customer_ID.unique())) # Get our unique customers
products = list(grouped_purchased.Product_ID.unique()) # Get our unique products that were purchased
quantity = list(grouped_purchased.Quantity) # All of our purchases

rows = grouped_purchased.Customer_ID.astype(CategoricalDtype(categories = customers)).cat.codes 
# Get the associated row indices
cols = grouped_purchased.Product_ID.astype(CategoricalDtype(categories = products)).cat.codes 
# Get the associated column indices
purchases_sparse = sparse.csr_matrix((quantity, (rows, cols)), shape=(len(customers), len(products)))

In [15]:
purchases_sparse

<18484x158 sparse matrix of type '<class 'numpy.intc'>'
	with 59051 stored elements in Compressed Sparse Row format>

We have 18484 customers with 158 items. For these user/item interactions, 59051 of these items had a purchase. In terms of sparsity of the matrix, that makes:

In [16]:
matrix_size = purchases_sparse.shape[0]*purchases_sparse.shape[1] # Number of possible interactions in the matrix
num_purchases = len(purchases_sparse.nonzero()[0]) # Number of items interacted with
sparsity = 100*(1 - (num_purchases/matrix_size))
sparsity

97.97803231806365

97.98% of the interaction matrix is sparse, so we should be able to get decent results because we are well below the maximum sparsity.

## Creating a Training and Validation Set

In [17]:
import random

In [18]:
def make_train(ratings, pct_test = 0.2):
    '''
    This function will take in the original user-item matrix and "mask" a percentage of the original ratings where a
    user-item interaction has taken place for use as a test set. The test set will contain all of the original ratings, 
    while the training set replaces the specified percentage of them with a zero in the original ratings matrix. 
    
    parameters: 
    
    ratings - the original ratings matrix from which you want to generate a train/test set. Test is just a complete
    copy of the original set. This is in the form of a sparse csr_matrix. 
    
    pct_test - The percentage of user-item interactions where an interaction took place that you want to mask in the 
    training set for later comparison to the test set, which contains all of the original ratings. 
    
    returns:
    
    training_set - The altered version of the original data with a certain percentage of the user-item pairs 
    that originally had interaction set back to zero.
    
    test_set - A copy of the original ratings matrix, unaltered, so it can be used to see how the rank order 
    compares with the actual interactions.
    
    user_inds - From the randomly selected user-item indices, which user rows were altered in the training data.
    This will be necessary later when evaluating the performance via AUC.
    '''
    test_set = ratings.copy() # Make a copy of the original set to be the test set. 
    test_set[test_set != 0] = 1 # Store the test set as a binary preference matrix
    training_set = ratings.copy() # Make a copy of the original data we can alter as our training set. 
    nonzero_inds = training_set.nonzero() # Find the indices in the ratings data where an interaction exists
    nonzero_pairs = list(zip(nonzero_inds[0], nonzero_inds[1])) # Zip these pairs together of user,item index into list
    random.seed(0) # Set the random seed to zero for reproducibility
    num_samples = int(np.ceil(pct_test*len(nonzero_pairs))) # Round the number of samples needed to the nearest integer
    samples = random.sample(nonzero_pairs, num_samples) # Sample a random number of user-item pairs without replacement
    user_inds = [index[0] for index in samples] # Get the user row indices
    item_inds = [index[1] for index in samples] # Get the item column indices
    training_set[user_inds, item_inds] = 0 # Assign all of the randomly chosen user-item pairs to zero
    training_set.eliminate_zeros() # Get rid of zeros in sparse array storage after update to save space
    return training_set, test_set, list(set(user_inds)) # Output the unique list of user rows that were altered  

In [19]:
product_train, product_test, product_users_altered = make_train(purchases_sparse, pct_test = 0.2)

## Implementing Alternating Least Squares(ALS) for Implicit Feedback

In [20]:
def implicit_weighted_ALS(training_set, lambda_val = 0.1, alpha = 40, iterations = 10, rank_size = 20, seed = 0):
    '''
    Implicit weighted ALS taken from Hu, Koren, and Volinsky 2008. Designed for alternating least squares and implicit
    feedback based collaborative filtering. 
    
    parameters:
    
    training_set - Our matrix of ratings with shape m x n, where m is the number of users and n is the number of items.
    Should be a sparse csr matrix to save space. 
    
    lambda_val - Used for regularization during alternating least squares. Increasing this value may increase bias
    but decrease variance. Default is 0.1. 
    
    alpha - The parameter associated with the confidence matrix discussed in the paper, where Cui = 1 + alpha*Rui. 
    The paper found a default of 40 most effective. Decreasing this will decrease the variability in confidence between
    various ratings.
    
    iterations - The number of times to alternate between both user feature vector and item feature vector in
    alternating least squares. More iterations will allow better convergence at the cost of increased computation. 
    The authors found 10 iterations was sufficient, but more may be required to converge. 
    
    rank_size - The number of latent features in the user/item feature vectors. The paper recommends varying this 
    between 20-200. Increasing the number of features may overfit but could reduce bias. 
    
    seed - Set the seed for reproducible results
    
    returns:
    
    The feature vectors for users and items. The dot product of these feature vectors should give you the expected 
    "rating" at each point in your original matrix. 
    '''
    
    # first set up our confidence matrix
    
    conf = (alpha*training_set) # To allow the matrix to stay sparse, I will add one later when each row is taken 
                                # and converted to dense. 
    num_user = conf.shape[0]
    num_item = conf.shape[1] # Get the size of our original ratings matrix, m x n
    
    # initialize our X/Y feature vectors randomly with a set seed
    rstate = np.random.RandomState(seed)
    
    X = sparse.csr_matrix(rstate.normal(size = (num_user, rank_size))) # Random numbers in a m x rank shape
    Y = sparse.csr_matrix(rstate.normal(size = (num_item, rank_size))) # Normally this would be rank x n but we can 
                                                                 # transpose at the end. Makes calculation more simple.
    X_eye = sparse.eye(num_user)
    Y_eye = sparse.eye(num_item)
    lambda_eye = lambda_val * sparse.eye(rank_size) # Our regularization term lambda*I. 
    
    # We can compute this before iteration starts. 
    
    # Begin iterations
   
    for iter_step in range(iterations): # Iterate back and forth between solving X given fixed Y and vice versa
        # Compute yTy and xTx at beginning of each iteration to save computing time
        yTy = Y.T.dot(Y)
        xTx = X.T.dot(X)
        # Being iteration to solve for X based on fixed Y
        for u in range(num_user):
            conf_samp = conf[u,:].toarray() # Grab user row from confidence matrix and convert to dense
            pref = conf_samp.copy() 
            pref[pref != 0] = 1 # Create binarized preference vector 
            CuI = sparse.diags(conf_samp, [0]) # Get Cu - I term, don't need to subtract 1 since we never added it 
            yTCuIY = Y.T.dot(CuI).dot(Y) # This is the yT(Cu-I)Y term 
            yTCupu = Y.T.dot(CuI + Y_eye).dot(pref.T) # This is the yTCuPu term, where we add the eye back in
                                                      # Cu - I + I = Cu
            X[u] = spsolve(yTy + yTCuIY + lambda_eye, yTCupu) 
            # Solve for Xu = ((yTy + yT(Cu-I)Y + lambda*I)^-1)yTCuPu, equation 4 from the paper  
        # Begin iteration to solve for Y based on fixed X 
        for i in range(num_item):
            conf_samp = conf[:,i].T.toarray() # transpose to get it in row format and convert to dense
            pref = conf_samp.copy()
            pref[pref != 0] = 1 # Create binarized preference vector
            CiI = sparse.diags(conf_samp, [0]) # Get Ci - I term, don't need to subtract 1 since we never added it
            xTCiIX = X.T.dot(CiI).dot(X) # This is the xT(Cu-I)X term
            xTCiPi = X.T.dot(CiI + X_eye).dot(pref.T) # This is the xTCiPi term
            Y[i] = spsolve(xTx + xTCiIX + lambda_eye, xTCiPi)
            # Solve for Yi = ((xTx + xT(Cu-I)X) + lambda*I)^-1)xTCiPi, equation 5 from the paper
    # End iterations
    return X, Y.T # Transpose at the end to make up for not being transposed at the beginning. 
                         # Y needs to be rank x n. Keep these as separate matrices for scale reasons. 

In [21]:
user_vecs, item_vecs = implicit_weighted_ALS(product_train, lambda_val = 0.1, alpha = 15, iterations = 1,
                                            rank_size = 20)

In [22]:
user_vecs[0,:].dot(item_vecs).toarray()[0,:5]

array([5.24299219e-02, 9.85570586e-05, 1.25361284e-02, 3.89815739e-02,
       7.83943702e-03])

## Speeding Up ALS

In [23]:
import implicit
from implicit.als import AlternatingLeastSquares

In [24]:
alpha = 15
user_vecs, item_vecs = implicit.alternating_least_squares((product_train*alpha).astype('double'), 
                                                          factors=20, 
                                                          regularization = 0.1, 
                                                         iterations = 50)

This method is deprecated. Please use the AlternatingLeastSquares class instead


## Evaluating the Recommender System

In [25]:
from sklearn import metrics

In [26]:
def auc_score(predictions, test):
    '''
    This simple function will output the area under the curve using sklearn's metrics. 
    
    parameters:
    
    - predictions: your prediction output
    
    - test: the actual target result you are comparing to
    
    returns:
    
    - AUC (area under the Receiver Operating Characterisic curve)
    '''
    fpr, tpr, thresholds = metrics.roc_curve(test, predictions)
    return metrics.auc(fpr, tpr)

In [27]:
def calc_mean_auc(training_set, altered_users, predictions, test_set):
    '''
    This function will calculate the mean AUC by user for any user that had their user-item matrix altered. 
    
    parameters:
    
    training_set - The training set resulting from make_train, where a certain percentage of the original
    user/item interactions are reset to zero to hide them from the model 
    
    predictions - The matrix of your predicted ratings for each user/item pair as output from the implicit MF.
    These should be stored in a list, with user vectors as item zero and item vectors as item one. 
    
    altered_users - The indices of the users where at least one user/item pair was altered from make_train function
    
    test_set - The test set constucted earlier from make_train function
    
    
    
    returns:
    
    The mean AUC (area under the Receiver Operator Characteristic curve) of the test set only on user-item interactions
    there were originally zero to test ranking ability in addition to the most popular items as a benchmark.
    '''
    
    
    store_auc = [] # An empty list to store the AUC for each user that had an item removed from the training set
    popularity_auc = [] # To store popular AUC scores
    pop_items = np.array(test_set.sum(axis = 0)).reshape(-1) # Get sum of item iteractions to find most popular
    item_vecs = predictions[1]
    for user in altered_users: # Iterate through each user that had an item altered
        training_row = training_set[user,:].toarray().reshape(-1) # Get the training set row
        zero_inds = np.where(training_row == 0) # Find where the interaction had not yet occurred
        # Get the predicted values based on our user/item vectors
        user_vec = predictions[0][user,:]
        pred = user_vec.dot(item_vecs).toarray()[0,zero_inds].reshape(-1)
        # Get only the items that were originally zero
        # Select all ratings from the MF prediction for this user that originally had no iteraction
        actual = test_set[user,:].toarray()[0,zero_inds].reshape(-1) 
        # Select the binarized yes/no interaction pairs from the original full data
        # that align with the same pairs in training 
        pop = pop_items[zero_inds] # Get the item popularity for our chosen items
        store_auc.append(auc_score(pred, actual)) # Calculate AUC for the given user and store
        popularity_auc.append(auc_score(pop, actual)) # Calculate AUC using most popular and score
    # End users iteration
    
    return float('%.3f'%np.mean(store_auc)), float('%.3f'%np.mean(popularity_auc))  
   # Return the mean AUC rounded to three decimal places for both test and popularity benchmark

In [28]:
calc_mean_auc(product_train, product_users_altered, 
              [sparse.csr_matrix(user_vecs), sparse.csr_matrix(item_vecs.T)], product_test)
# AUC for our recommender system

(0.772, 0.836)

Our systems mean AUC is a bit lower than the Benchmark if we want to improve the scores we can go back to change our parameters

### An Example using our recommendation engine

In [29]:
# Retrieving the products already purchased by a customer

customers_arr = np.array(customers) # Array of customer_IDs from the ratings matrix
products_arr = np.array(products) # Array of product_IDs from the ratings matrix

In [30]:
def get_items_purchased(Customer_ID, mf_train, customers_list, products_list, item_lookup):
    '''
    This just tells me which items have been already purchased by a specific user in the training set. 
    
    parameters: 
    
    customer_id - Input the customer's id number that you want to see prior purchases of at least once
    
    mf_train - The initial ratings training set used (without weights applied)
    
    customers_list - The array of customers used in the ratings matrix
    
    products_list - The array of products used in the ratings matrix
    
    item_lookup - A simple pandas dataframe of the unique product ID/product descriptions available
    
    returns:
    
    A list of item IDs and item descriptions for a particular customer that were already purchased in the training set
    '''
    cust_ind = np.where(customers_list == Customer_ID)[0][0] # Returns the index row of our customer id
    purchased_ind = mf_train[cust_ind,:].nonzero()[1] # Get column indices of purchased items
    prod_codes = products_list[purchased_ind] # Get the stock codes for our purchased items
    return item_lookup.loc[item_lookup.Product_ID.isin(prod_codes)]

In [31]:
customers_arr[:5]

array([11000, 11001, 11002, 11003, 11004], dtype=int64)

In [32]:
# examining customer 11000 purchases from the training set.
get_items_purchased(11000, product_train, customers_arr, products_arr, item_lookup)

,Product_ID,Product,Product_Name
5432,353,Mountain-200,"Mountain-200 Silver, 38"
5442,214,Sport-100,"Sport-100 Helmet, Red"
5457,541,Touring Tire,Touring Tire
5458,530,Touring Tire Tube,Touring Tire Tube
5459,573,Touring-1000,"Touring-1000 Blue, 46"
5476,485,Fender Set - Mountain,Fender Set - Mountain


In [33]:
from sklearn.preprocessing import MinMaxScaler

In [34]:
def rec_items(customer_id, mf_train, user_vecs, item_vecs, customer_list, item_list, item_lookup, num_items = 10):
    '''
    This function will return the top recommended items to our users 
    
    parameters:
    
    customer_id - Input the customer's id number that you want to get recommendations for
    
    mf_train - The training matrix you used for matrix factorization fitting
    
    user_vecs - the user vectors from your fitted matrix factorization
    
    item_vecs - the item vectors from your fitted matrix factorization
    
    customer_list - an array of the customer's ID numbers that make up the rows of your ratings matrix 
                    (in order of matrix)
    
    item_list - an array of the products that make up the columns of your ratings matrix
                    (in order of matrix)
    
    item_lookup - A simple pandas dataframe of the unique product ID/product descriptions available
    
    num_items - The number of items you want to recommend in order of best recommendations. Default is 10. 
    
    returns:
    
    - The top n recommendations chosen based on the user/item vectors for items never interacted with/purchased
    '''
    
    cust_ind = np.where(customer_list == customer_id)[0][0] # Returns the index row of our customer id
    pref_vec = mf_train[cust_ind,:].toarray() # Get the ratings from the training set ratings matrix
    pref_vec = pref_vec.reshape(-1) + 1 # Add 1 to everything, so that items not purchased yet become equal to 1
    pref_vec[pref_vec > 1] = 0 # Make everything already purchased zero
    rec_vector = user_vecs[cust_ind,:].dot(item_vecs.T) # Get dot product of user vector and all item vectors
    # Scale this recommendation vector between 0 and 1
    min_max = MinMaxScaler()
    rec_vector_scaled = min_max.fit_transform(rec_vector.reshape(-1,1))[:,0] 
    recommend_vector = pref_vec*rec_vector_scaled 
    # Items already purchased have their recommendation multiplied by zero
    product_idx = np.argsort(recommend_vector)[::-1][:num_items] # Sort the indices of the items into order 
    # of best recommendations
    rec_list = [] # start empty list to store items
    for index in product_idx:
        code = item_list[index]
        rec_list.append([code, item_lookup.Product.loc[item_lookup.Product_ID == code].iloc[0], item_lookup.Product_Name.loc[item_lookup.Product_ID == code].iloc[0]]) 
        # Append our descriptions to the list
    codes = [item[0] for item in rec_list]
    Products = [item[1] for item in rec_list]
    Description = [item[2] for item in rec_list]
    final_frame = pd.DataFrame({'Product_ID': codes, 'Product': Products, 'Product_Name': Description}) # Create a dataframe 
    return final_frame[['Product_ID', 'Product', 'Product_Name']] # Switch order of columns around

In [35]:
# Products recommended by our model for Customer 11000
rec_items(11000, product_train, user_vecs, item_vecs, customers_arr, products_arr, item_lookup,
                       num_items = 3)

,Product_ID,Product,Product_Name
0,362,Mountain-200,"Mountain-200 Black, 46"
1,358,Mountain-200,"Mountain-200 Black, 38"
2,467,Half-Finger Gloves,"Half-Finger Gloves, L"


### CHOOSING A RANDOM LIST OF 10 CUSTOMERS TO RECOMMEND PRODUCTS FOR THEM USING OUR MODEL

In [36]:
ID_11000 = rec_items(11000, product_train, user_vecs, item_vecs, customers_arr, products_arr, item_lookup,
                       num_items = 3)
# create a dataframe
metrics = ['Recommendation','2nd_Recommendation','3rd_Recommendation']

# Create an array with the score values
scores = np.array(ID_11000)

# Create the dataframe
a = pd.DataFrame(scores, columns = ['Product_ID','11000','Product_Name'], index = metrics).reset_index()
a1 = a['Product_ID']
a1 = pd.DataFrame(a1)
a2 = a['Product_Name']
a2 = pd.DataFrame(a2)
a.drop(['Product_ID','Product_Name'],axis=1,inplace=True)
a

,index,11000
0,Recommendation,Mountain-200
1,2nd_Recommendation,Mountain-200
2,3rd_Recommendation,Half-Finger Gloves


In [37]:
get_items_purchased(11005, product_train, customers_arr, products_arr, item_lookup)

,Product_ID,Product,Product_Name
5453,537,HL Mountain Tire,HL Mountain Tire
5454,528,Mountain Tire Tube,Mountain Tire Tube
5551,480,Patch kit,Patch Kit/8 Patches


In [38]:
rec_items(11005, product_train, user_vecs, item_vecs, customers_arr, products_arr, item_lookup,
                       num_items = 3)

,Product_ID,Product,Product_Name
0,536,ML Mountain Tire,ML Mountain Tire
1,484,Bike Wash,Bike Wash - Dissolver
2,535,LL Mountain Tire,LL Mountain Tire


In [39]:
ID_11005 = rec_items(11005, product_train, user_vecs, item_vecs, customers_arr, products_arr, item_lookup,
                       num_items = 3)
scores = np.array(ID_11005)
b = pd.DataFrame(scores, columns = ['Product_ID','11005','Product_Name'], index = metrics).reset_index()
b1 = b['Product_ID']
b1 = pd.DataFrame(b1)
b2 = b['Product_Name']
b2 = pd.DataFrame(b2)
b.drop(['Product_ID','Product_Name'],axis=1,inplace=True)
b

,index,11005
0,Recommendation,ML Mountain Tire
1,2nd_Recommendation,Bike Wash
2,3rd_Recommendation,LL Mountain Tire


In [40]:
get_items_purchased(11010, product_train, customers_arr, products_arr, item_lookup)

,Product_ID,Product,Product_Name
1,346,Mountain-100,"Mountain-100 Silver, 44"
5513,361,Mountain-200,"Mountain-200 Black, 42"
5661,575,Touring-1000,"Touring-1000 Blue, 54"


In [41]:
ID_11010 = rec_items(11010, product_train, user_vecs, item_vecs, customers_arr, products_arr, item_lookup,
                       num_items = 3)
scores = np.array(ID_11010)
c = pd.DataFrame(scores, columns = ['Product_ID','11010','Product_Name'], index = metrics).reset_index()
c1 = c['Product_ID']
c1 = pd.DataFrame(c1)
c2 = c['Product_Name']
c2 = pd.DataFrame(c2)
c.drop(['Product_ID','Product_Name'],axis=1,inplace=True)
c

,index,11010
0,Recommendation,Mountain-200
1,2nd_Recommendation,Mountain-200
2,3rd_Recommendation,Road-150


In [42]:
ID_11015 = rec_items(11015, product_train, user_vecs, item_vecs, customers_arr, products_arr, item_lookup,
                       num_items = 3)
scores = np.array(ID_11015)
d = pd.DataFrame(scores, columns = ['Product_ID','11015','Product_Name'], index = metrics).reset_index()
d1 = d['Product_ID']
d1 = pd.DataFrame(d1)
d2 = d['Product_Name']
d2 = pd.DataFrame(d2)
d.drop(['Product_ID','Product_Name'],axis=1,inplace=True)
d

,index,11015
0,Recommendation,Patch kit
1,2nd_Recommendation,Road Tire Tube
2,3rd_Recommendation,LL Road Tire


In [43]:
ID_11020 = rec_items(11020, product_train, user_vecs, item_vecs, customers_arr, products_arr, item_lookup,
                       num_items = 3)
scores = np.array(ID_11020)
e = pd.DataFrame(scores, columns = ['Product_ID','11020','Product_Name'], index = metrics).reset_index()
e1 = e['Product_ID']
e1 = pd.DataFrame(e1)
e2 = e['Product_Name']
e2 = pd.DataFrame(e2)
e.drop(['Product_ID','Product_Name'],axis=1,inplace=True)
e

,index,11020
0,Recommendation,Mountain-200
1,2nd_Recommendation,Mountain Bottle Cage
2,3rd_Recommendation,Hitch Rack - 4-Bike


In [44]:
ID_11025 = rec_items(11025, product_train, user_vecs, item_vecs, customers_arr, products_arr, item_lookup,
                       num_items = 3)
scores = np.array(ID_11025)
f = pd.DataFrame(scores, columns = ['Product_ID','11025','Product_Name'], index = metrics).reset_index()
f1 = f['Product_ID']
f1 = pd.DataFrame(f1)
f2 = f['Product_Name']
f2 = pd.DataFrame(f2)
f.drop(['Product_ID','Product_Name'],axis=1,inplace=True)
f

,index,11025
0,Recommendation,Cycling Cap
1,2nd_Recommendation,Short-Sleeve Classic Jersey
2,3rd_Recommendation,Mountain-200


In [45]:
ID_11030 = rec_items(11030, product_train, user_vecs, item_vecs, customers_arr, products_arr, item_lookup,
                       num_items = 3)
scores = np.array(ID_11030)
g = pd.DataFrame(scores, columns = ['Product_ID','11030','Product_Name'], index = metrics).reset_index()
g1 = g['Product_ID']
g1 = pd.DataFrame(g1)
g2 = g['Product_Name']
f2 = pd.DataFrame(g2)
g.drop(['Product_ID','Product_Name'],axis=1,inplace=True)
g

,index,11030
0,Recommendation,Fender Set - Mountain
1,2nd_Recommendation,Long-Sleeve Logo Jersey
2,3rd_Recommendation,Road-550-W


In [46]:
ID_11035 = rec_items(11035, product_train, user_vecs, item_vecs, customers_arr, products_arr, item_lookup,
                       num_items = 3)
scores = np.array(ID_11035)
h = pd.DataFrame(scores, columns = ['Product_ID','11035','Product_Name'], index = metrics).reset_index()
h1 = h['Product_ID']
h1 = pd.DataFrame(h1)
h2 = h['Product_Name']
h2 = pd.DataFrame(h2)
h.drop(['Product_ID','Product_Name'],axis=1,inplace=True)
h

,index,11035
0,Recommendation,Sport-100
1,2nd_Recommendation,Mountain-200
2,3rd_Recommendation,Road-550-W


In [47]:
ID_11040 = rec_items(11040, product_train, user_vecs, item_vecs, customers_arr, products_arr, item_lookup,
                       num_items = 3)
scores = np.array(ID_11040)
i = pd.DataFrame(scores, columns = ['Product_ID','11040','Product_Name'], index = metrics).reset_index()
i1 = i['Product_ID']
i1 = pd.DataFrame(i1)
i2 = i['Product_Name']
i2 = pd.DataFrame(i2)
i.drop(['Product_ID','Product_Name'],axis=1,inplace=True)
i

,index,11040
0,Recommendation,Mountain-200
1,2nd_Recommendation,Racing Socks
2,3rd_Recommendation,Mountain-200


In [48]:
ID_11045 = rec_items(11045, product_train, user_vecs, item_vecs, customers_arr, products_arr, item_lookup,
                       num_items = 3)
ID_11045.index
scores = np.array(ID_11045)
j = pd.DataFrame(scores, columns = ['Product_ID','11045','Product_Name'], index = metrics).reset_index()
j1 = j['Product_ID']
j1 = pd.DataFrame(j1)
j2 = j['Product_Name']
j2 = pd.DataFrame(j2)
j.drop(['Product_ID','Product_Name'],axis=1,inplace=True)
j

,index,11045
0,Recommendation,Half-Finger Gloves
1,2nd_Recommendation,Long-Sleeve Logo Jersey
2,3rd_Recommendation,Half-Finger Gloves


In [49]:
dd=pd.concat([a1,b1,c1,d1,e1,f1,g1,h1,i1,j1.reindex(a1.index)], axis=1)
dd=dd.transpose()
dd=dd.reset_index()
dd=dd.rename(columns={'index': 'Product_ID'})
dd.drop('Product_ID', axis=1, inplace = True)
dd.rename(columns={0: 'Product_ID', 1: 'Product_ID2', 2: 'Product_ID3'},inplace = True)
dd

,Product_ID,Product_ID2,Product_ID3
0,362,358,467
1,536,484,535
2,354,359,310
3,480,529,538
4,355,478,483
5,225,491,359
6,485,231,384
7,217,353,384
8,361,482,358
9,465,231,467


In [50]:
aa=pd.concat([a2,b2,c2,d2,e2,f2,g2,h2,i2,j2.reindex(a2.index)], axis=1)
aa=aa.transpose()
aa=aa.reset_index()
aa=aa.rename(columns={'index': 'Product_Name'})
aa.drop('Product_Name', axis=1, inplace = True)
aa.rename(columns={0: 'Product_Name', 1: '2nd_Product_Name', 2: '3rd_Product_Name'},inplace = True)
aa

,Product_Name,2nd_Product_Name,3rd_Product_Name
0,"Mountain-200 Black, 46","Mountain-200 Black, 38","Half-Finger Gloves, L"
1,ML Mountain Tire,Bike Wash - Dissolver,LL Mountain Tire
2,"Mountain-200 Silver, 42","Mountain-200 Black, 38","Road-150 Red, 62"
3,Patch Kit/8 Patches,Road Tire Tube,LL Road Tire
4,"Mountain-200 Silver, 42",Mountain Bottle Cage,Hitch Rack - 4-Bike
5,Fender Set - Mountain,"Long-Sleeve Logo Jersey, M","Road-550-W Yellow, 40"
6,Fender Set - Mountain,"Long-Sleeve Logo Jersey, M","Road-550-W Yellow, 40"
7,"Sport-100 Helmet, Black","Mountain-200 Silver, 38","Road-550-W Yellow, 40"
8,"Mountain-200 Black, 42","Racing Socks, L","Mountain-200 Black, 38"
9,"Half-Finger Gloves, M","Long-Sleeve Logo Jersey, M","Half-Finger Gloves, L"


In [51]:
from functools import reduce

# Combine altogether
dfs = [a, b, c, d, e, f, g, h, i, j]
summary = reduce(lambda left,right: pd.merge(left, right, on = 'index'), dfs)
summary

,index,11000,11005,11010,11015,11020,11025,11030,11035,11040,11045
0,Recommendation,Mountain-200,ML Mountain Tire,Mountain-200,Patch kit,Mountain-200,Cycling Cap,Fender Set - Mountain,Sport-100,Mountain-200,Half-Finger Gloves
1,2nd_Recommendation,Mountain-200,Bike Wash,Mountain-200,Road Tire Tube,Mountain Bottle Cage,Short-Sleeve Classic Jersey,Long-Sleeve Logo Jersey,Mountain-200,Racing Socks,Long-Sleeve Logo Jersey
2,3rd_Recommendation,Half-Finger Gloves,LL Mountain Tire,Road-150,LL Road Tire,Hitch Rack - 4-Bike,Mountain-200,Road-550-W,Road-550-W,Mountain-200,Half-Finger Gloves


In [52]:
summary = summary.transpose()
summary.columns = summary.iloc[0]
summary.drop(summary.index[0], inplace = True)
summary.rename(columns = {'index' : 'Customer_ID'}, inplace = True)

summary

index,Recommendation,2nd_Recommendation,3rd_Recommendation
11000,Mountain-200,Mountain-200,Half-Finger Gloves
11005,ML Mountain Tire,Bike Wash,LL Mountain Tire
11010,Mountain-200,Mountain-200,Road-150
11015,Patch kit,Road Tire Tube,LL Road Tire
11020,Mountain-200,Mountain Bottle Cage,Hitch Rack - 4-Bike
11025,Cycling Cap,Short-Sleeve Classic Jersey,Mountain-200
11030,Fender Set - Mountain,Long-Sleeve Logo Jersey,Road-550-W
11035,Sport-100,Mountain-200,Road-550-W
11040,Mountain-200,Racing Socks,Mountain-200
11045,Half-Finger Gloves,Long-Sleeve Logo Jersey,Half-Finger Gloves


In [53]:
rec = summary.reset_index()
rec

index,index,Recommendation,2nd_Recommendation,3rd_Recommendation
0,11000,Mountain-200,Mountain-200,Half-Finger Gloves
1,11005,ML Mountain Tire,Bike Wash,LL Mountain Tire
2,11010,Mountain-200,Mountain-200,Road-150
3,11015,Patch kit,Road Tire Tube,LL Road Tire
4,11020,Mountain-200,Mountain Bottle Cage,Hitch Rack - 4-Bike
5,11025,Cycling Cap,Short-Sleeve Classic Jersey,Mountain-200
6,11030,Fender Set - Mountain,Long-Sleeve Logo Jersey,Road-550-W
7,11035,Sport-100,Mountain-200,Road-550-W
8,11040,Mountain-200,Racing Socks,Mountain-200
9,11045,Half-Finger Gloves,Long-Sleeve Logo Jersey,Half-Finger Gloves


In [54]:
rec = rec.rename(columns={'index': 'Customer_ID'})
rec

index,Customer_ID,Recommendation,2nd_Recommendation,3rd_Recommendation
0,11000,Mountain-200,Mountain-200,Half-Finger Gloves
1,11005,ML Mountain Tire,Bike Wash,LL Mountain Tire
2,11010,Mountain-200,Mountain-200,Road-150
3,11015,Patch kit,Road Tire Tube,LL Road Tire
4,11020,Mountain-200,Mountain Bottle Cage,Hitch Rack - 4-Bike
5,11025,Cycling Cap,Short-Sleeve Classic Jersey,Mountain-200
6,11030,Fender Set - Mountain,Long-Sleeve Logo Jersey,Road-550-W
7,11035,Sport-100,Mountain-200,Road-550-W
8,11040,Mountain-200,Racing Socks,Mountain-200
9,11045,Half-Finger Gloves,Long-Sleeve Logo Jersey,Half-Finger Gloves


**These are the 3 Recommended Products for our Random Customers**

In [55]:
# Adding back the product_IDs so we can get revenue and profit of products later

myD=pd.concat([rec,dd.reindex(rec.index)], axis=1)
myA=myD[['Customer_ID','Product_ID','Recommendation', 'Product_ID2','2nd_Recommendation','Product_ID3','3rd_Recommendation']]
myA

,Customer_ID,Product_ID,Recommendation,Product_ID2,2nd_Recommendation,Product_ID3,3rd_Recommendation
0,11000,362,Mountain-200,358,Mountain-200,467,Half-Finger Gloves
1,11005,536,ML Mountain Tire,484,Bike Wash,535,LL Mountain Tire
2,11010,354,Mountain-200,359,Mountain-200,310,Road-150
3,11015,480,Patch kit,529,Road Tire Tube,538,LL Road Tire
4,11020,355,Mountain-200,478,Mountain Bottle Cage,483,Hitch Rack - 4-Bike
5,11025,225,Cycling Cap,491,Short-Sleeve Classic Jersey,359,Mountain-200
6,11030,485,Fender Set - Mountain,231,Long-Sleeve Logo Jersey,384,Road-550-W
7,11035,217,Sport-100,353,Mountain-200,384,Road-550-W
8,11040,361,Mountain-200,482,Racing Socks,358,Mountain-200
9,11045,465,Half-Finger Gloves,231,Long-Sleeve Logo Jersey,467,Half-Finger Gloves


In [56]:
myB=pd.concat([myA,aa.reindex(myA.index)], axis=1)
rec=myB[['Customer_ID','Product_ID','Recommendation','Product_Name', 'Product_ID2','2nd_Recommendation','2nd_Product_Name','Product_ID3','3rd_Recommendation','3rd_Product_Name']]
rec

,Customer_ID,Product_ID,Recommendation,Product_Name,Product_ID2,2nd_Recommendation,2nd_Product_Name,Product_ID3,3rd_Recommendation,3rd_Product_Name
0,11000,362,Mountain-200,"Mountain-200 Black, 46",358,Mountain-200,"Mountain-200 Black, 38",467,Half-Finger Gloves,"Half-Finger Gloves, L"
1,11005,536,ML Mountain Tire,ML Mountain Tire,484,Bike Wash,Bike Wash - Dissolver,535,LL Mountain Tire,LL Mountain Tire
2,11010,354,Mountain-200,"Mountain-200 Silver, 42",359,Mountain-200,"Mountain-200 Black, 38",310,Road-150,"Road-150 Red, 62"
3,11015,480,Patch kit,Patch Kit/8 Patches,529,Road Tire Tube,Road Tire Tube,538,LL Road Tire,LL Road Tire
4,11020,355,Mountain-200,"Mountain-200 Silver, 42",478,Mountain Bottle Cage,Mountain Bottle Cage,483,Hitch Rack - 4-Bike,Hitch Rack - 4-Bike
5,11025,225,Cycling Cap,Fender Set - Mountain,491,Short-Sleeve Classic Jersey,"Long-Sleeve Logo Jersey, M",359,Mountain-200,"Road-550-W Yellow, 40"
6,11030,485,Fender Set - Mountain,Fender Set - Mountain,231,Long-Sleeve Logo Jersey,"Long-Sleeve Logo Jersey, M",384,Road-550-W,"Road-550-W Yellow, 40"
7,11035,217,Sport-100,"Sport-100 Helmet, Black",353,Mountain-200,"Mountain-200 Silver, 38",384,Road-550-W,"Road-550-W Yellow, 40"
8,11040,361,Mountain-200,"Mountain-200 Black, 42",482,Racing Socks,"Racing Socks, L",358,Mountain-200,"Mountain-200 Black, 38"
9,11045,465,Half-Finger Gloves,"Half-Finger Gloves, M",231,Long-Sleeve Logo Jersey,"Long-Sleeve Logo Jersey, M",467,Half-Finger Gloves,"Half-Finger Gloves, L"


In [57]:
rec.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Customer_ID         10 non-null     object
 1   Product_ID          10 non-null     object
 2   Recommendation      10 non-null     object
 3   Product_Name        10 non-null     object
 4   Product_ID2         10 non-null     object
 5   2nd_Recommendation  10 non-null     object
 6   2nd_Product_Name    10 non-null     object
 7   Product_ID3         10 non-null     object
 8   3rd_Recommendation  10 non-null     object
 9   3rd_Product_Name    10 non-null     object
dtypes: object(10)
memory usage: 928.0+ bytes


In [58]:
rec[['Customer_ID','Product_ID','Product_ID2','Product_ID3']] = rec[['Customer_ID','Product_ID','Product_ID2','Product_ID3']].astype(str).astype(int)
rec.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Customer_ID         10 non-null     int32 
 1   Product_ID          10 non-null     int32 
 2   Recommendation      10 non-null     object
 3   Product_Name        10 non-null     object
 4   Product_ID2         10 non-null     int32 
 5   2nd_Recommendation  10 non-null     object
 6   2nd_Product_Name    10 non-null     object
 7   Product_ID3         10 non-null     int32 
 8   3rd_Recommendation  10 non-null     object
 9   3rd_Product_Name    10 non-null     object
dtypes: int32(4), object(6)
memory usage: 768.0+ bytes


In [59]:
# seperating each recommendations in other to get product price and profit

recommend_1 = rec[['Customer_ID','Product_ID','Recommendation','Product_Name']]
recommend_2 = rec[['Customer_ID','Product_ID2','2nd_Recommendation','2nd_Product_Name']]
recommend_3 = rec[['Customer_ID','Product_ID3','3rd_Recommendation','3rd_Product_Name']]

In [60]:
recommend_1

,Customer_ID,Product_ID,Recommendation,Product_Name
0,11000,362,Mountain-200,"Mountain-200 Black, 46"
1,11005,536,ML Mountain Tire,ML Mountain Tire
2,11010,354,Mountain-200,"Mountain-200 Silver, 42"
3,11015,480,Patch kit,Patch Kit/8 Patches
4,11020,355,Mountain-200,"Mountain-200 Silver, 42"
5,11025,225,Cycling Cap,Fender Set - Mountain
6,11030,485,Fender Set - Mountain,Fender Set - Mountain
7,11035,217,Sport-100,"Sport-100 Helmet, Black"
8,11040,361,Mountain-200,"Mountain-200 Black, 42"
9,11045,465,Half-Finger Gloves,"Half-Finger Gloves, M"


## Importing Price and Cost data for products

In [61]:
conn = pyodbc.connect('Driver={SQL Server};'
'Server=DESKTOP-90M94KF;'
'Database=AdventureWorksDW2019;'
'Trusted_Connection=yes;')

In [62]:
mydata=pd.read_sql_query("""
SELECT DISTINCT
DP.ProductKey AS Product_ID
,[EnglishProductName] AS Product_Name
,DP.ModelName AS Recommendation
,FIS.[UnitPrice]
,FIS.ProductStandardCost


FROM [dbo].[FactInternetSales] AS FIS

LEFT JOIN [dbo].[DimProduct] AS DP

ON FIS.ProductKey = DP.ProductKey
""", conn)

conn.close()

mydata

,Product_ID,Product_Name,Recommendation,UnitPrice,ProductStandardCost
0,237,"Long-Sleeve Logo Jersey, XL",Long-Sleeve Logo Jersey,49.9900,38.4923
1,342,"Road-650 Black, 52",Road-650,699.0982,413.1463
2,343,"Road-650 Black, 52",Road-650,782.9900,486.7066
3,369,"Road-250 Red, 48",Road-250,2443.3500,1518.7864
4,467,"Half-Finger Gloves, L",Half-Finger Gloves,24.4900,9.1593
...,...,...,...,...,...
153,575,"Touring-1000 Blue, 54",Touring-1000,2384.0700,1481.9379
154,582,"Road-350-W Yellow, 44",Road-350-W,1700.9900,1082.5100
155,583,"Road-350-W Yellow, 48",Road-350-W,1700.9900,1082.5100
156,598,"Mountain-500 Black, 44",Mountain-500,539.9900,294.5797


In [63]:
mydata2 = mydata.copy()
mydata2 = mydata2.rename(columns={'Product_ID':'Product_ID2', 'Product_Name':'2nd_Product_Name', 'Recommendation':'Recommendation2', 'UnitPrice':'UnitPrice2', 'ProductStandardCost':'ProductStandardCost2'})
mydata2

,Product_ID2,2nd_Product_Name,Recommendation2,UnitPrice2,ProductStandardCost2
0,237,"Long-Sleeve Logo Jersey, XL",Long-Sleeve Logo Jersey,49.9900,38.4923
1,342,"Road-650 Black, 52",Road-650,699.0982,413.1463
2,343,"Road-650 Black, 52",Road-650,782.9900,486.7066
3,369,"Road-250 Red, 48",Road-250,2443.3500,1518.7864
4,467,"Half-Finger Gloves, L",Half-Finger Gloves,24.4900,9.1593
...,...,...,...,...,...
153,575,"Touring-1000 Blue, 54",Touring-1000,2384.0700,1481.9379
154,582,"Road-350-W Yellow, 44",Road-350-W,1700.9900,1082.5100
155,583,"Road-350-W Yellow, 48",Road-350-W,1700.9900,1082.5100
156,598,"Mountain-500 Black, 44",Mountain-500,539.9900,294.5797


In [64]:
mydata3 = mydata.copy()
mydata3 = mydata3.rename(columns={'Product_ID':'Product_ID3', 'Product_Name':'3rd_Product_Name', 'Recommendation':'Recommendation3', 'UnitPrice':'UnitPrice3', 'ProductStandardCost':'ProductStandardCost3'})
mydata3

,Product_ID3,3rd_Product_Name,Recommendation3,UnitPrice3,ProductStandardCost3
0,237,"Long-Sleeve Logo Jersey, XL",Long-Sleeve Logo Jersey,49.9900,38.4923
1,342,"Road-650 Black, 52",Road-650,699.0982,413.1463
2,343,"Road-650 Black, 52",Road-650,782.9900,486.7066
3,369,"Road-250 Red, 48",Road-250,2443.3500,1518.7864
4,467,"Half-Finger Gloves, L",Half-Finger Gloves,24.4900,9.1593
...,...,...,...,...,...
153,575,"Touring-1000 Blue, 54",Touring-1000,2384.0700,1481.9379
154,582,"Road-350-W Yellow, 44",Road-350-W,1700.9900,1082.5100
155,583,"Road-350-W Yellow, 48",Road-350-W,1700.9900,1082.5100
156,598,"Mountain-500 Black, 44",Mountain-500,539.9900,294.5797


### Merging the price and cost to our Recommendations

In [65]:
A_recommendation = pd.merge(recommend_1, mydata[['Product_ID','UnitPrice','ProductStandardCost']], on = ['Product_ID'])
B_recommendation = pd.merge(recommend_2, mydata2[['Product_ID2','UnitPrice2','ProductStandardCost2']], on = ['Product_ID2'])
C_recommendation = pd.merge(recommend_3, mydata3[['Product_ID3','UnitPrice3','ProductStandardCost3']], on = ['Product_ID3'])
C_recommendation

,Customer_ID,Product_ID3,3rd_Recommendation,3rd_Product_Name,UnitPrice3,ProductStandardCost3
0,11000,467,Half-Finger Gloves,"Half-Finger Gloves, L",24.4900,9.1593
1,11045,467,Half-Finger Gloves,"Half-Finger Gloves, L",24.4900,9.1593
2,11005,535,LL Mountain Tire,LL Mountain Tire,24.9900,9.3463
3,11010,310,Road-150,"Road-150 Red, 62",3578.2700,2171.2942
4,11015,538,LL Road Tire,LL Road Tire,21.4900,8.0373
5,11020,483,Hitch Rack - 4-Bike,Hitch Rack - 4-Bike,120.0000,44.8800
6,11025,359,Mountain-200,"Road-550-W Yellow, 40",2294.9900,1251.9813
7,11030,384,Road-550-W,"Road-550-W Yellow, 40",1120.4900,713.0798
8,11035,384,Road-550-W,"Road-550-W Yellow, 40",1120.4900,713.0798
9,11040,358,Mountain-200,"Mountain-200 Black, 38",2049.0982,1105.8100


In [66]:
My_Rec = pd.merge(B_recommendation,C_recommendation, on = ['Customer_ID'])
My_Rec

,Customer_ID,Product_ID2,2nd_Recommendation,2nd_Product_Name,UnitPrice2,ProductStandardCost2,Product_ID3,3rd_Recommendation,3rd_Product_Name,UnitPrice3,ProductStandardCost3
0,11000,358,Mountain-200,"Mountain-200 Black, 38",2049.0982,1105.8100,467,Half-Finger Gloves,"Half-Finger Gloves, L",24.4900,9.1593
1,11005,484,Bike Wash,Bike Wash - Dissolver,7.9500,2.9733,535,LL Mountain Tire,LL Mountain Tire,24.9900,9.3463
2,11010,359,Mountain-200,"Mountain-200 Black, 38",2294.9900,1251.9813,310,Road-150,"Road-150 Red, 62",3578.2700,2171.2942
3,11015,529,Road Tire Tube,Road Tire Tube,3.9900,1.4923,538,LL Road Tire,LL Road Tire,21.4900,8.0373
4,11020,478,Mountain Bottle Cage,Mountain Bottle Cage,9.9900,3.7363,483,Hitch Rack - 4-Bike,Hitch Rack - 4-Bike,120.0000,44.8800
5,11025,491,Short-Sleeve Classic Jersey,"Long-Sleeve Logo Jersey, M",53.9900,41.5723,359,Mountain-200,"Road-550-W Yellow, 40",2294.9900,1251.9813
6,11030,231,Long-Sleeve Logo Jersey,"Long-Sleeve Logo Jersey, M",49.9900,38.4923,384,Road-550-W,"Road-550-W Yellow, 40",1120.4900,713.0798
7,11045,231,Long-Sleeve Logo Jersey,"Long-Sleeve Logo Jersey, M",49.9900,38.4923,467,Half-Finger Gloves,"Half-Finger Gloves, L",24.4900,9.1593
8,11035,353,Mountain-200,"Mountain-200 Silver, 38",2319.9900,1265.6195,384,Road-550-W,"Road-550-W Yellow, 40",1120.4900,713.0798
9,11040,482,Racing Socks,"Racing Socks, L",8.9900,3.3623,358,Mountain-200,"Mountain-200 Black, 38",2049.0982,1105.8100


In [67]:
# price and cost of each item recommended

My_Recommendation = pd.merge(A_recommendation,My_Rec, on = ['Customer_ID'])
My_Recommendation

,Customer_ID,Product_ID,Recommendation,Product_Name,UnitPrice,ProductStandardCost,Product_ID2,2nd_Recommendation,2nd_Product_Name,UnitPrice2,ProductStandardCost2,Product_ID3,3rd_Recommendation,3rd_Product_Name,UnitPrice3,ProductStandardCost3
0,11000,362,Mountain-200,"Mountain-200 Black, 46",2049.0982,1105.8100,358,Mountain-200,"Mountain-200 Black, 38",2049.0982,1105.8100,467,Half-Finger Gloves,"Half-Finger Gloves, L",24.4900,9.1593
1,11005,536,ML Mountain Tire,ML Mountain Tire,29.9900,11.2163,484,Bike Wash,Bike Wash - Dissolver,7.9500,2.9733,535,LL Mountain Tire,LL Mountain Tire,24.9900,9.3463
2,11010,354,Mountain-200,"Mountain-200 Silver, 42",2071.4196,1117.8559,359,Mountain-200,"Mountain-200 Black, 38",2294.9900,1251.9813,310,Road-150,"Road-150 Red, 62",3578.2700,2171.2942
3,11015,480,Patch kit,Patch Kit/8 Patches,2.2900,0.8565,529,Road Tire Tube,Road Tire Tube,3.9900,1.4923,538,LL Road Tire,LL Road Tire,21.4900,8.0373
4,11020,355,Mountain-200,"Mountain-200 Silver, 42",2319.9900,1265.6195,478,Mountain Bottle Cage,Mountain Bottle Cage,9.9900,3.7363,483,Hitch Rack - 4-Bike,Hitch Rack - 4-Bike,120.0000,44.8800
5,11025,225,Cycling Cap,Fender Set - Mountain,8.9900,6.9223,491,Short-Sleeve Classic Jersey,"Long-Sleeve Logo Jersey, M",53.9900,41.5723,359,Mountain-200,"Road-550-W Yellow, 40",2294.9900,1251.9813
6,11030,485,Fender Set - Mountain,Fender Set - Mountain,21.9800,8.2205,231,Long-Sleeve Logo Jersey,"Long-Sleeve Logo Jersey, M",49.9900,38.4923,384,Road-550-W,"Road-550-W Yellow, 40",1120.4900,713.0798
7,11035,217,Sport-100,"Sport-100 Helmet, Black",34.9900,13.0863,353,Mountain-200,"Mountain-200 Silver, 38",2319.9900,1265.6195,384,Road-550-W,"Road-550-W Yellow, 40",1120.4900,713.0798
8,11040,361,Mountain-200,"Mountain-200 Black, 42",2294.9900,1251.9813,482,Racing Socks,"Racing Socks, L",8.9900,3.3623,358,Mountain-200,"Mountain-200 Black, 38",2049.0982,1105.8100
9,11045,465,Half-Finger Gloves,"Half-Finger Gloves, M",24.4900,9.1593,231,Long-Sleeve Logo Jersey,"Long-Sleeve Logo Jersey, M",49.9900,38.4923,467,Half-Finger Gloves,"Half-Finger Gloves, L",24.4900,9.1593


In [68]:
# Calculating profit of items recommended

My_Recommendation['Total_Price'] = My_Recommendation['UnitPrice'] + My_Recommendation['UnitPrice2'] + My_Recommendation['UnitPrice3']
My_Recommendation['Total_Cost'] = My_Recommendation['ProductStandardCost'] + My_Recommendation['ProductStandardCost2'] + My_Recommendation['ProductStandardCost3']
My_Recommendation['Total_Profit'] = My_Recommendation['Total_Price'] - My_Recommendation['Total_Cost']
My_Recommendation

,Customer_ID,Product_ID,Recommendation,Product_Name,UnitPrice,ProductStandardCost,Product_ID2,2nd_Recommendation,2nd_Product_Name,UnitPrice2,ProductStandardCost2,Product_ID3,3rd_Recommendation,3rd_Product_Name,UnitPrice3,ProductStandardCost3,Total_Price,Total_Cost,Total_Profit
0,11000,362,Mountain-200,"Mountain-200 Black, 46",2049.0982,1105.8100,358,Mountain-200,"Mountain-200 Black, 38",2049.0982,1105.8100,467,Half-Finger Gloves,"Half-Finger Gloves, L",24.4900,9.1593,4122.6864,2220.7793,1901.9071
1,11005,536,ML Mountain Tire,ML Mountain Tire,29.9900,11.2163,484,Bike Wash,Bike Wash - Dissolver,7.9500,2.9733,535,LL Mountain Tire,LL Mountain Tire,24.9900,9.3463,62.9300,23.5359,39.3941
2,11010,354,Mountain-200,"Mountain-200 Silver, 42",2071.4196,1117.8559,359,Mountain-200,"Mountain-200 Black, 38",2294.9900,1251.9813,310,Road-150,"Road-150 Red, 62",3578.2700,2171.2942,7944.6796,4541.1314,3403.5482
3,11015,480,Patch kit,Patch Kit/8 Patches,2.2900,0.8565,529,Road Tire Tube,Road Tire Tube,3.9900,1.4923,538,LL Road Tire,LL Road Tire,21.4900,8.0373,27.7700,10.3861,17.3839
4,11020,355,Mountain-200,"Mountain-200 Silver, 42",2319.9900,1265.6195,478,Mountain Bottle Cage,Mountain Bottle Cage,9.9900,3.7363,483,Hitch Rack - 4-Bike,Hitch Rack - 4-Bike,120.0000,44.8800,2449.9800,1314.2358,1135.7442
5,11025,225,Cycling Cap,Fender Set - Mountain,8.9900,6.9223,491,Short-Sleeve Classic Jersey,"Long-Sleeve Logo Jersey, M",53.9900,41.5723,359,Mountain-200,"Road-550-W Yellow, 40",2294.9900,1251.9813,2357.9700,1300.4759,1057.4941
6,11030,485,Fender Set - Mountain,Fender Set - Mountain,21.9800,8.2205,231,Long-Sleeve Logo Jersey,"Long-Sleeve Logo Jersey, M",49.9900,38.4923,384,Road-550-W,"Road-550-W Yellow, 40",1120.4900,713.0798,1192.4600,759.7926,432.6674
7,11035,217,Sport-100,"Sport-100 Helmet, Black",34.9900,13.0863,353,Mountain-200,"Mountain-200 Silver, 38",2319.9900,1265.6195,384,Road-550-W,"Road-550-W Yellow, 40",1120.4900,713.0798,3475.4700,1991.7856,1483.6844
8,11040,361,Mountain-200,"Mountain-200 Black, 42",2294.9900,1251.9813,482,Racing Socks,"Racing Socks, L",8.9900,3.3623,358,Mountain-200,"Mountain-200 Black, 38",2049.0982,1105.8100,4353.0782,2361.1536,1991.9246
9,11045,465,Half-Finger Gloves,"Half-Finger Gloves, M",24.4900,9.1593,231,Long-Sleeve Logo Jersey,"Long-Sleeve Logo Jersey, M",49.9900,38.4923,467,Half-Finger Gloves,"Half-Finger Gloves, L",24.4900,9.1593,98.9700,56.8109,42.1591


### Profit on the Recommended Products of 10 Random customers

In [69]:
My_Recommendation_Profit = My_Recommendation.copy()
My_Recommendation_Profit.drop(['Product_ID','Product_Name','UnitPrice','ProductStandardCost','Product_ID2','2nd_Product_Name','UnitPrice2','ProductStandardCost2','Product_ID3','3rd_Product_Name','UnitPrice3','ProductStandardCost3'], axis=1, inplace = True)
My_Recommendation_Profit

,Customer_ID,Recommendation,2nd_Recommendation,3rd_Recommendation,Total_Price,Total_Cost,Total_Profit
0,11000,Mountain-200,Mountain-200,Half-Finger Gloves,4122.6864,2220.7793,1901.9071
1,11005,ML Mountain Tire,Bike Wash,LL Mountain Tire,62.9300,23.5359,39.3941
2,11010,Mountain-200,Mountain-200,Road-150,7944.6796,4541.1314,3403.5482
3,11015,Patch kit,Road Tire Tube,LL Road Tire,27.7700,10.3861,17.3839
4,11020,Mountain-200,Mountain Bottle Cage,Hitch Rack - 4-Bike,2449.9800,1314.2358,1135.7442
5,11025,Cycling Cap,Short-Sleeve Classic Jersey,Mountain-200,2357.9700,1300.4759,1057.4941
6,11030,Fender Set - Mountain,Long-Sleeve Logo Jersey,Road-550-W,1192.4600,759.7926,432.6674
7,11035,Sport-100,Mountain-200,Road-550-W,3475.4700,1991.7856,1483.6844
8,11040,Mountain-200,Racing Socks,Mountain-200,4353.0782,2361.1536,1991.9246
9,11045,Half-Finger Gloves,Long-Sleeve Logo Jersey,Half-Finger Gloves,98.9700,56.8109,42.1591


In [70]:
print('$',My_Recommendation_Profit['Total_Price'].sum())

$ 26085.994199999997


In [71]:
print('$',My_Recommendation_Profit['Total_Profit'].sum())

$ 11505.907099999999


## Estimated Total Revenue/Profit for recommended products to all customers if purchased

In [72]:
# Total Revenue estimate
TR = statistics.mean(My_Recommendation_Profit['Total_Price'])
TOTAL_REVENUE =TR*18484
print('$',TOTAL_REVENUE)

$ 48217351.67927999


In [73]:
# Total Profit estimate
TP = statistics.mean(My_Recommendation_Profit['Total_Profit'])
TOTAL_PROFIT=TP*18484
print('$',TOTAL_PROFIT)

$ 21267518.683639996
